In [1]:
import numpy as np
import os
from pathlib import Path
import glob
import pandas as pd
from collections import Counter
import math
from tqdm import tqdm
import subprocess as sub

In [2]:
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
DATA_PATH = Path('/content/drive/MyDrive/Data_Shortcut/Module1_ADNI/')

In [5]:

os.listdir(Path(DATA_PATH, 'Zipped_data'))

['ADNI_base1.zip',
 'ADNI',
 'ADNI_base2.zip',
 'ADNI_base4.zip',
 'ADNI_base3.zip',
 'ADNI_base5.zip',
 'AD_ADNI23_Base.zip',
 'AD_AllTimepoints_FBB.zip',
 'CN_AllTimepoints_FBB1.zip',
 'CN_AllTimepoints_FBB.zip',
 'CN_AllTimepoints_FBB2.zip',
 'CN_AllTimepoints_FBB3.zip',
 'CN_AllTimepoints_FBB4.zip']

In [10]:
!unzip /content/drive/MyDrive/Data_Shortcut/Module1_ADNI/Zipped_data/CN_AllTimepoints_FBB4.zip -d '/content/drive/MyDrive/Data_Shortcut/Module1_ADNI/Zipped_data/'

Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/MyDrive/Data_Shortcut/Module1_ADNI/Zipped_data/ADNI/006_S_6375/ADNI3_florbetaben_4X5_FBB/2020-09-16_13_02_21.0/I1346121/ADNI_006_S_6375_PT_ADNI3_florbetaben_4X5__br_raw_20201006141417188_59_S964701_I1346121.dcm  
  inflating: /content/drive/MyDrive/Data_Shortcut/Module1_ADNI/Zipped_data/ADNI/006_S_6375/ADNI3_florbetaben_4X5_FBB/2020-09-16_13_02_21.0/I1346121/ADNI_006_S_6375_PT_ADNI3_florbetaben_4X5__br_raw_20201006141417204_58_S964701_I1346121.dcm  
  inflating: /content/drive/MyDrive/Data_Shortcut/Module1_ADNI/Zipped_data/ADNI/006_S_6375/ADNI3_florbetaben_4X5_FBB/2020-09-16_13_02_21.0/I1346121/ADNI_006_S_6375_PT_ADNI3_florbetaben_4X5__br_raw_20201006141417220_57_S964701_I1346121.dcm  
  inflating: /content/drive/MyDrive/Data_Shortcut/Module1_ADNI/Zipped_data/ADNI/006_S_6375/ADNI3_florbetaben_4X5_FBB/2020-09-16_13_02_21.0/I1346121/ADNI_006_S_6375_PT_ADNI3_florbetaben_4X5__br_raw_20201006141417236_56_S964701_

In [ ]:
unzip /content/drive/MyDrive/Data_Shortcut/Module1_ADNI/MRI/MRI_T2w1.zip -d '/content/drive/MyDrive/Data_Shortcut/Module1_ADNI/MRI/'

Archive:  /content/drive/MyDrive/Data_Shortcut/Module1_ADNI/MRI/MRI_T2w1.zip
replace /content/drive/MyDrive/Data_Shortcut/Module1_ADNI/MRI/ADNI/100_S_0035/ADNI_STUDY_Double_TSE2s/2005-10-20_11_19_13.0/I8122/ADNI_100_S_0035_MR_ADNI_STUDY_Double_TSE2s__br_raw_20051021103525116_2_S9958_I8122.dcm? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Just want to see how much additional information I can get from merging the PET data together: 

In [11]:
os.listdir(DATA_PATH)

['PET', 'Zipped_data', 'CSVS', 'ADNI_CIDS', 'MRI']

In [12]:
patient_list_PET_folder = os.listdir(DATA_PATH/'PET')

In [13]:
patient_list_Zip_folder = os.listdir(DATA_PATH/'Zipped_data'/'ADNI')
patient_list_Zip_folder[:2]

['023_S_1190', '023_S_0042']

In [14]:
## Yes = patients in both 
## No = patients in Zip but not in PET folder 
pt_in_PET = []
for patient_id in patient_list_Zip_folder: 
  if patient_id in patient_list_PET_folder:
    pt_in_PET.append("yes")
  else: 
    pt_in_PET.append('no')
Counter(pt_in_PET)

Counter({'no': 272, 'yes': 582})

In [29]:
DATA_PATH

PosixPath('/content/drive/MyDrive/Data_Shortcut/Module1_ADNI')

In [40]:
PET_list_Zip_folder = ['/'.join(x.split('/')[-2:]) for x in glob.glob(str(Path(DATA_PATH,'Zipped_data','ADNI', '*', '*')))]
PET_list_Zip_folder[:2]

['023_S_1190/AV45_Coreg,_Avg,_Standardized_Image_and_Voxel_Size',
 '023_S_1190/AV45_Co-registered,_Averaged']

In [41]:
PET_list_PET_folder = ['/'.join(x.split('/')[-2:]) for x in glob.glob(str(Path(DATA_PATH,'PET', '*', '*')))]
PET_list_PET_folder[:2]

['023_S_4501/AV45_Co-registered_Dynamic',
 '023_S_4501/AV45_Coreg,_Avg,_Standardized_Image_and_Voxel_Size']

In [42]:
## Yes = patients in both 
## No = patients in ZIP but not in PET folder 
acq_in_Zip = []
acq_not_in_PET = []

for acq in PET_list_Zip_folder: 
  if acq in PET_list_PET_folder:
    acq_in_Zip.append("yes")
  else: 
    acq_in_Zip.append('no')
    acq_not_in_PET.append(acq)
Counter(acq_in_Zip)

Counter({'no': 270, 'yes': 706})

In [39]:
len(acq_not_in_PET)

270

In [43]:
acq = acq_not_in_PET[0]

In [44]:
ZIP_PATH = Path(DATA_PATH,'Zipped_data','ADNI')
PET_PATH = Path(DATA_PATH,'PET',)

In [45]:
os.listdir(Path(ZIP_PATH, acq))

['2017-05-30_13_57_34.0', '2019-07-25_14_54_13.0']

In [49]:
str(Path(ZIP_PATH, acq))

'/content/drive/MyDrive/Data_Shortcut/Module1_ADNI/Zipped_data/ADNI/022_S_6013/ADNI_Brain_PET__Raw_FBB'

In [48]:
str(PET_PATH)+'/'+acq.split('/')[0]+'/.'

'/content/drive/MyDrive/Data_Shortcut/Module1_ADNI/PET/022_S_6013/.'

In [51]:
str(PET_PATH)+'/'+acq.split('/')[0]

'/content/drive/MyDrive/Data_Shortcut/Module1_ADNI/PET/022_S_6013'

In [52]:
for acq in acq_not_in_PET: 
  command = 'mv '+str(Path(ZIP_PATH, acq))+' '+str(PET_PATH)+'/'+acq.split('/')[0]+'/.'
  if acq.split('/')[0] in os.listdir(PET_PATH): 
    try: 
      !{command}
    except: 
      print('failed for: ', acq)
  else: 
    os.makedirs(str(PET_PATH)+'/'+acq.split('/')[0], exist_ok = True)
    try: 
      !{command}
    except: 
      print('failed for: ', acq)

In [54]:
PET_list_Zip_folder = ['/'.join(x.split('/')[-2:]) for x in glob.glob(str(Path(DATA_PATH,'Zipped_data','ADNI', '*', '*')))]
PET_list_PET_folder = ['/'.join(x.split('/')[-2:]) for x in glob.glob(str(Path(DATA_PATH,'PET', '*', '*')))]
acq_in_Zip = []
acq_not_in_PET = []
for acq in PET_list_Zip_folder: 
  if acq in PET_list_PET_folder:
    acq_in_Zip.append("yes")
  else: 
    acq_in_Zip.append('no')
    acq_not_in_PET.append(acq)
Counter(acq_in_Zip)

Counter({'yes': 706})

In [55]:
## Now going to remove the duplicated folders or merge them? 
duplicated_patient_ids = []
for patient_id in os.listdir(Path(DATA_PATH, 'PET')): 
  if '(' in patient_id: 
    if patient_id[:-4] in os.listdir(Path(DATA_PATH, 'PET')): 
      duplicated_patient_ids.append(patient_id)
      

In [56]:
for patient_id in duplicated_patient_ids: 
  first_set = set(os.listdir(Path(DATA_PATH,'PET', patient_id)))
  second_set = set(os.listdir(Path(DATA_PATH, 'PET',patient_id[:-4])))
  if first_set != second_set: 
    # print(patient_id)
    print('elts in first set, not in second: ', [x for x in first_set if x not in second_set])
    print('elts in second set, not in first: ', [x for x in second_set if x not in first_set])
    command = 'mv '+str(Path(DATA_PATH,'PET', patient_id[:-4], ))+'\\ \\'+patient_id[-3:-1]+'\\'+patient_id[-1:]+'/* '+str(Path(DATA_PATH, 'PET', patient_id[:-4]))+'/.'
    # print(command)
    try: 
      !{command}
    except: 
      print('Failure for: ', command)

In [57]:
## Look to see the duplicated patient IDs have stuff in them 
duplicated_patient_ids = []
for patient_id in os.listdir(Path(DATA_PATH, 'PET')): 
  if '(' in patient_id:
    print(patient_id) 
    if len(os.listdir(Path(DATA_PATH, 'PET', patient_id))) > 0: 
      print(patient_id)
      print(os.listdir(Path(DATA_PATH, 'PET', patient_id)))
    else: 
      command = 'rm -r '+str(Path(DATA_PATH,'PET', patient_id[:-4], ))+'\\ \\'+patient_id[-3:-1]+'\\'+patient_id[-1:]
      print(command)
      try: 
        !{command}
      except: 
        print('it didnt work')

003_S_4354 (1)
003_S_4354 (1)
['AV45_Coreg,_Avg,_Std_Img_and_Vox_Siz,_Uniform_Resolution']


In [58]:
## Now going to remove the duplicated folders or merge them? 
duplicated_patient_ids = []
for patient_id in os.listdir(Path(DATA_PATH, 'PET')): 
  if '(' in patient_id: 
    if patient_id[:-4] in os.listdir(Path(DATA_PATH, 'PET')): 
      duplicated_patient_ids.append(patient_id)
      

In [62]:
Path(DATA_PATH, "PET")

PosixPath('/content/drive/MyDrive/Data_Shortcut/Module1_ADNI/PET')

In [63]:
Counter([x.split('/')[-1] for x in glob.glob(str(Path(DATA_PATH, 'PET', '*', '*', )))]).most_common()

[('AV45_Coreg,_Avg,_Std_Img_and_Vox_Siz,_Uniform_Resolution', 645),
 ('AV45_Co-registered,_Averaged', 639),
 ('AV45_Coreg,_Avg,_Standardized_Image_and_Voxel_Size', 634),
 ('AV45_Co-registered_Dynamic', 633),
 ('AV45-Early_Coreg,_Dyn,_Std_Img_and_Vox_Siz,_Uniform_Resolution', 35),
 ('AV45-Early_Coreg,_Avg,_Standardized_Image_and_Voxel_Size', 33),
 ('AV45-Early_Coreg,_Avg,_Std_Img_and_Vox_Siz,_Uniform_Resolution', 32),
 ('AV45-Early_Coreg,_Dyn,_Standardized_Image_and_Voxel_Size', 30),
 ('ADNI_Brain_PET__Raw_FBB', 30),
 ('ADNI3-BRAIN_FBB', 20),
 ('PET_AC_FBB', 18),
 ('ADNI_Brain_PET__Raw_AV45', 16),
 ('ADNI_AV45_Brain_20min_dyn', 12),
 ('Dyn_Brain_VPHD_6i16s_AC_FBB', 12),
 ('90-110_min_dynamic-3D_4i_16s_FBB', 12),
 ('Florbetaben_20_MIN_DYN_FBB', 11),
 ('ADNI3__florbetaben_Brain_20min_dyn_FBB', 10),
 ('PET_Brain_FBB', 9),
 ('3D_FLORBETABEN_ADNI3___4i_16s_FBB', 9),
 ('NEURACEQ_3D_Brain_AC_ADNI_FBB', 8),
 ('AV45_Co-registered_Dynamic (1)', 7),
 ('AV45_Co-registered,_Averaged (1)', 7),
 ('Bra

In [ ]:
glob.glob(str(Path(DATA_PATH, '*', '*', '037_S_4015', )))

['/content/drive/MyDrive/Data_Shortcut/Module1_ADNI/Zipped_data/ADNI/037_S_4015']